In [2]:
import pandas as pd

In [1]:
from datasets import load_dataset
dataset = load_dataset("stsb_multi_mt", name="es", split="train")

/home/manuel.couto.pintos/Documents/code/ludosym/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
iterator = iter(dataset)
for _ in range(10): print(next(iterator)) 

{'sentence1': 'Un avión está despegando.', 'sentence2': 'Un avión está despegando.', 'similarity_score': 5.0}
{'sentence1': 'Un hombre está tocando una gran flauta.', 'sentence2': 'Un hombre está tocando una flauta.', 'similarity_score': 3.799999952316284}
{'sentence1': 'Un hombre está untando queso rallado en una pizza.', 'sentence2': 'Un hombre está untando queso rallado en una pizza cruda.', 'similarity_score': 3.799999952316284}
{'sentence1': 'Tres hombres están jugando al ajedrez.', 'sentence2': 'Dos hombres están jugando al ajedrez.', 'similarity_score': 2.5999999046325684}
{'sentence1': 'Un hombre está tocando el violonchelo.', 'sentence2': 'Un hombre sentado está tocando el violonchelo.', 'similarity_score': 4.25}
{'sentence1': 'Algunos hombres están luchando.', 'sentence2': 'Dos hombres están luchando.', 'similarity_score': 4.25}
{'sentence1': 'Un hombre está fumando.', 'sentence2': 'Un hombre está patinando.', 'similarity_score': 0.5}
{'sentence1': 'El hombre está tocando el 

In [3]:
import pandas as pd

df = pd.DataFrame(dataset)

In [4]:
df.similarity_score = df.similarity_score.apply(lambda x: x/5)

In [5]:
df

,sentence1,sentence2,similarity_score
0,Un avión está despegando.,Un avión está despegando.,1.00
1,Un hombre está tocando una gran flauta.,Un hombre está tocando una flauta.,0.76
2,Un hombre está untando queso rallado en una pi...,Un hombre está untando queso rallado en una pi...,0.76
3,Tres hombres están jugando al ajedrez.,Dos hombres están jugando al ajedrez.,0.52
4,Un hombre está tocando el violonchelo.,Un hombre sentado está tocando el violonchelo.,0.85
...,...,...,...
5744,Vendavales severos mientras la tormenta Clodag...,Merkel promete la solidaridad de la OTAN con L...,0.00
5745,Docenas de egipcios rehenes tomados por terror...,El número de muertos en el accidente de un bar...,0.00
5746,El Presidente se dirige a Bahrein,Presidente Xi: China seguirá ayudando a combat...,0.00
5747,China y la India se comprometen a fomentar los...,China lucha por tranquilizar a los nerviosos c...,0.00


In [3]:
from dotenv import load_dotenv
import os
import sys

load_dotenv()

sys.path.append("..")

In [113]:
from plugins.estimators import Mistral_large

In [8]:
import nltk
from tqdm import tqdm

import re
WORD = re.compile(r'\w+')
def regTokenize(text):
    words = WORD.findall(text)
    return words

In [9]:
from tqdm.auto import tqdm
tqdm.pandas()

df['sent1_len'] = df.sentence1.progress_apply(lambda x: len(regTokenize(str(x))))
df['sent2_len'] = df.sentence2.progress_apply(lambda x: len(regTokenize(str(x))))


100%|██████████| 5749/5749 [00:00<00:00, 447931.60it/s]


In [10]:
df.sent1_len.sum() + df.sent2_len.sum()

140370

In [106]:
system = dict(role='system', content='Te pasaré una lista de 10 textos. Devuelve otra lista de 10 textos. Reescribe un único texto nuevo por cada texto que te envíe. Parafrasea cada uno asegurandote de que el texto generado sea distinto al original. Asegurate de conservar significado. Separa cada uno de los textos generados con \n\n. No quites ni añadas información. Usa sinónimos en la medida de lo posible. En total generarás 10 textos. Uno por cada texto original.')

In [114]:
model = Mistral_large()

In [13]:
response = model.model.send([system, dict(role='user', content=df.iloc[0].sentence1)])

index=0 message=ChatMessage(role='assistant', content='Un aeroplano se encuentra en proceso de despegar.', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>


In [14]:
response[-1]['content']

'Un aeroplano se encuentra en proceso de despegar.'

In [15]:
df.iloc[0]

sentence1           Un avión está despegando.
sentence2           Un avión está despegando.
similarity_score                          1.0
sent1_len                                   4
sent2_len                                   4
Name: 0, dtype: object

## Get first augmentation

In [42]:
import numpy as np

all_responses = []
for idx, batch in tqdm(df.groupby(np.arange(len(df))//10)):
    response = model.model.send([system]+batch.sentence1.apply(lambda x: dict(role='user', content=x)).values.tolist())
    all_responses.append(response)
    

  0%|          | 1/575 [00:04<42:36,  4.45s/it]

index=0 message=ChatMessage(role='assistant', content='Un avión se encuentra en pleno despegue.\n\nUn individuo se dedica a interpretar una gran flauta.\n\nUn hombre se encuentra esparciendo queso rallado sobre una pizza.\n\nTres hombres se hallan disputando una partida de ajedrez.\n\nUn hombre se dedica a interpretar el violonchelo.\n\nAlgunos hombres se encuentran luchando en una pelea.\n\nUn hombre se encuentra fumando un cigarrillo.\n\nEl hombre se dedica a interpretar el piano.\n\nUn hombre se encuentra tocando la guitarra y cantando al mismo tiempo.\n\nUna persona se encuentra lanzando a un gato hacia el techo.', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>


  0%|          | 2/575 [00:08<38:22,  4.02s/it]

index=0 message=ChatMessage(role='assistant', content='El individuo asestó un golpe a otro con un bastón.\n\nUna dama recoge y abraza a un joey de canguro.\n\nUn hombre está interpretando una flauta.\n\nUna persona está plegando un trozo de papel.\n\nUn varón está corriendo por el camino.\n\nUn perro está intentando quitarse el beicon de encima.\n\nEl oso ártico se está deslizando sobre la nieve.\n\nUna mujer está redactando.\n\nUn gato se está restregando contra el rostro del infante.\n\nEl hombre está montando en un caballo.', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>


  1%|          | 3/575 [00:12<37:30,  3.93s/it]

...

index=0 message=ChatMessage(role='assistant', content='Se lamenta Corea del Sur por la negativa de Corea del Norte a entablar diálogo\n\nLa localización del accidente aéreo fue suspendida durante la noche\n\nObama indaga a Turnbull sobre el convenio del puerto chino\n\nLa oferta benevolente de China a 1MDB ha generado significativos beneficios en Malasia\n\nHezbolá apunta a la congregación de Al Qaeda a lo largo de la frontera siria\n\nCorea del Sur informa sobre siete nuevos contagios de MERS\n\nObama solicita un frente global contra ISIS\n\nEl ejército estadounidense afirma realizar ataques aéreos contra ISIS en Siria e Irak\n\n¿Quién dejó abandonados 3 aviones Boeing en el aeropuerto?\n\nCorea del Norte llevará a juicio a un ciudadano de los Estados Unidos', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>


100%|█████████▉| 574/575 [1:01:19<00:06,  6.12s/it]

index=0 message=ChatMessage(role='assistant', content='Los empleados públicos de Hong Kong vuelven a sus labores mientras las manifestaciones menguan.\n\nReino Unido: El personal del NHS entró en estado de alarma tras los supuestos casos de Ébola...\n\nEl dirigente británico indica su respaldo a los ataques aéreos contra el IS en Siria.\n\nCorea del Norte acepta los diálogos con el Sur.\n\nIrán, un socio clave para Rusia: Novak.\n\nEl presidente de Taiwán alquila un avión para rendir homenaje a Lee.\n\nDos palestinos fallecen en atentados contra israelíes antes de la visita de Kerry.\n\nThomas Cook es acusado de priorizar los costes sobre los clientes.\n\nLa imagen "editada digitalmente" de la visita de Narendra Modi a las inundaciones de Chennai se vuelve viral.\n\nEl secretario general de la ONU celebra las elecciones presidenciales pacíficas en Guinea.', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>


100%|██████████| 575/575 [1:01:24<00:00,  6.41s/it]

index=0 message=ChatMessage(role='assistant', content='Las ejecuciones en Pakistán provocan indignación a nivel mundial\n\nDos ciudadanos australianos fallecen en Kenia tras el choque de un autobús contra un río\n\nFrancia clausura una mezquita y detiene a un individuo en medio de la represión posterior a los atentados\n\nSe dice que un avión ruso se ha estrellado en territorio egipcio\n\nFuertes ráfagas de viento mientras la tormenta Clodagh azota el Reino Unido\n\nDecenas de egipcios son tomados como rehenes por terroristas libios en represalia por los ataques aéreos\n\nEl Presidente se dirige a Bahrein\n\nChina y la India se comprometen a fortalecer sus relaciones bilaterales\n\nEl portavoz de Putin: Los cargos por dopaje parecen carecer de fundamento', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>


In [51]:
aux_list = list(map(lambda x: x[-1]['content'].split("\n\n"), all_responses))

In [71]:
from rich import print
retrived_texts = []
for (idx, batch), result in tqdm(zip(df.groupby(np.arange(len(df))//10), all_responses)):
    aux1 = batch.sentence1.tolist()
    aux2 = result[-1]['content'].split("\n\n")
    if len(aux1) == len(aux2):
        retrived_texts.append(aux2)
    else:
        retrived_texts.append([None]*10)
        
    

575it [00:00, 33103.53it/s]


In [72]:
import itertools

df['aug1'] = list(itertools.chain(*retrived_texts))

In [109]:
import time
sub_df = df[['sentence1', 'aug1']].loc[df['aug1'].isna()]
sub_df_responses = []

system_msg = dict(
    role='system', 
    content='Reescribe un único texto nuevo por cada texto que te envíe parafraseando al original. El texto generado debe ser distinto al original. Asegurate de conservar significado. No quites ni añadas información. Usa sinónimos en la medida de lo posible.')

for idx, batch in tqdm(sub_df.groupby(np.arange(len(sub_df))//10)):
    error = True
    while error:
        try:
            aux1 = batch.sentence1.apply(lambda x: model.model.send([system_msg, dict(role='user', content=x)])[-1]['content'])
            sub_df_responses.append(aux1)
            error = False
        except Exception as ex:
            error = True
            print(ex)


index=0 message=ChatMessage(role='assistant', content='El felino de pelaje blanco y negro está observando hacia la lente de la cámara.', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>
index=0 message=ChatMessage(role='assistant', content='Un felino de pelaje grisáceo y ojos esmeralda está observando hacia la lente.', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>
index=0 message=ChatMessage(role='assistant', content='Dos borregos se encuentran de pie sobre una loma cubierta de pasto.', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>
index=0 message=ChatMessage(role='assistant', content='Un caballero de atuendo blanco que lleva una botella de vino tinto y una dama con un jersey de cuello alto de color rojo y un chaleco blanco.', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>
index=0 message=ChatMessage(role='assistant', co

index=0 message=ChatMessage(role='assistant', content='Una vista cercana de un diminuto cactus en un tiesto.', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>
index=0 message=ChatMessage(role='assistant', content='Las personas que cometen pecados no alcanzan el paraíso.', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>
index=0 message=ChatMessage(role='assistant', content='¿Cuánto tiempo debe transcurrir después de la extirpación antes de que el menor regrese al colegio?', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>
index=0 message=ChatMessage(role='assistant', content='Dios Santo. Todos nosotros somos recipientes colmados de numerosas prodigiosidades.', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>
index=0 message=ChatMessage(role='assistant', content='¿Cómo fue la "reacción" en Chicago?\n\nLa respuesta en Chicago fue

index=0 message=ChatMessage(role='assistant', content='Empleo de la potencia en protección de la individuo.', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>
index=0 message=ChatMessage(role='assistant', content='El Departamento de Salud de China informó que otras cinco personas habían fallecido a causa del SARS y que 159 personas adicionales estaban contagiadas.', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>
index=0 message=ChatMessage(role='assistant', content="Los valores de McDonald's aumentaron $1.83, o un 8.3%, alcanzando un récord diario de $23.89 al cierre.\n\nLos precios de las acciones de McDonald's se elevaron en $1.83, lo que representa un 8.3%, para terminar el día con un máximo de $23.89.\n\nLas participaciones de McDonald's ascendieron $1.83, o un 8.3%, para concluir el día en un máximo de $23.89.\n\nEl precio de las acciones de McDonald's se incrementó en $1.83, un 8.3%, para establecer

index=0 message=ChatMessage(role='assistant', content='LO ÁNGELES - "Hulk" se convirtió en un gigante en la taquilla durante su fin de semana de estreno, logrando una cifra récord de apertura en junio de 62,6 millones de dólares.', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>
index=0 message=ChatMessage(role='assistant', content='Una serie de ataques aéreos en Iraq ha resultado en la trágica pérdida de 18 vidas y 108 personas heridas.\n\nUn estallido de bombardeos en Iraq ha causado la muerte de 18 personas y ha dejado a 108 heridas.\n\nLa ola de ataques con bombas en Iraq ha provocado la muerte de 18 personas y ha dejado a otras 108 heridas.\n\nUna racha de bombardeos en Iraq ha causado 18 muertes y 108 heridos.\n\nUna serie de ataques con bomba en Iraq ha dejado un saldo de 18 muertos y 108 heridos.\n\nLos bombardeos en Iraq han causado la muerte de 18 personas y han herido a otras 108.\n\nUna oleada de bombardeos en Iraq ha provocado la tr

index=0 message=ChatMessage(role='assistant', content='Se ha lanzado una advertencia de tsunami para la zona sur de Alaska.', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>
index=0 message=ChatMessage(role='assistant', content='Es necesario que revises el escrito antes del 4 de julio de 2012, que cae en miércoles.', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>


...

index=0 message=ChatMessage(role='assistant', content='El defensor chino Chen llega a Estados Unidos.', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>
index=0 message=ChatMessage(role='assistant', content='En Nueva Orleans, 19 personas resultaron lesionadas debido a un incidente de disparos.\n\nEl trágico suceso de un tiroteo en Nueva Orleans dejó un saldo de 19 personas heridas.\n\nUn total de 19 personas sufrieron heridas durante un tiroteo ocurrido en Nueva Orleans.\n\nEn Nueva Orleans, un tiroteo causó lesiones a 19 individuos.\n\nUn lamentable tiroteo en Nueva Orleans provocó que 19 personas quedaran heridas.', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>
index=0 message=ChatMessage(role='assistant', content='Hagel se muestra afligido por los cambios "políticos" en el fondo de defensa de Estados Unidos.', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>
ind

index=0 message=ChatMessage(role='assistant', content='Mandela permanecerá en el hospital durante la temporada navideña.', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>
index=0 message=ChatMessage(role='assistant', content='Trágico incidente vial en el suroeste de China resulta en el fallecimiento de diez personas y lesiones a otras cinco.', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>
index=0 message=ChatMessage(role='assistant', content='El gobierno militar de Tailandia está reuniendo efectivos de seguridad para reprimir las manifestaciones en Bangkok.', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>
index=0 message=ChatMessage(role='assistant', content='Aso se rehúsa a dimitir debido a los comentarios de índole nazi\n\nLos comentarios nazis no logran que Aso renuncie\n\nAso se resiste a abandonar el cargo a causa de los comentarios nazis\n\nAso no cede ant

index=0 message=ChatMessage(role='assistant', content='Doris Lessing, la ganadora del Premio Nobel, fallece a la edad de 94 años.\n\nLa escritora premiada con el Nobel, Doris Lessing, deja este mundo a los 94 años.\n\nDoris Lessing, la célebre autora galardonada con el Nobel, nos deja a los 94 años.\n\nA los 94 años, fallece la reconocida escritora y ganadora del Nobel, Doris Lessing.\n\nDoris Lessing, autora laureada con el Nobel, parte de este mundo a los 94 años.\n\nLa afamada escritora Doris Lessing, ganadora del Nobel, muere a los 94 años.\n\nA los 94 años, nos deja la ilustre escritora y premiada con el Nobel, Doris Lessing.\n\nLa escritora Doris Lessing, galardonada con el Nobel, fallece a los 94 años.\n\nDoris Lessing, la renombrada autora del Nobel, fallece a la edad de 94 años.\n\nA los 94 años, muere la escritora premiada con el Nobel, Doris Lessing.', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>
index=0 message=ChatMessage(role='a

100%|██████████| 12/12 [23:45<00:00, 118.82s/it]

index=0 message=ChatMessage(role='assistant', content='En París, los sirios se manifiestan en contra de Erdogan, mientras que los venezolanos expresan su respaldo a Siria.', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>


In [125]:
df['aug1'].loc[df['aug1'].isna()] = list(itertools.chain(*list(map(lambda x: x.tolist(), sub_df_responses))))

/tmp/ipykernel_2235509/1258589170.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['aug1'].loc[df['aug1'].isna()] = list(itertools.chain(*list(map(lambda x: x.tolist(), sub_df_responses))))


In [127]:
any(df['aug1'].isna())

False

In [128]:
df.to_csv('data/augmented_sts.csv')

In [129]:
df

,sentence1,sentence2,similarity_score,sent1_len,sent2_len,aug1
0,Un avión está despegando.,Un avión está despegando.,1.00,4,4,Un avión se encuentra en pleno despegue.
1,Un hombre está tocando una gran flauta.,Un hombre está tocando una flauta.,0.76,7,6,Un individuo se dedica a interpretar una gran ...
2,Un hombre está untando queso rallado en una pi...,Un hombre está untando queso rallado en una pi...,0.76,9,10,Un hombre se encuentra esparciendo queso ralla...
3,Tres hombres están jugando al ajedrez.,Dos hombres están jugando al ajedrez.,0.52,6,6,Tres hombres se hallan disputando una partida ...
4,Un hombre está tocando el violonchelo.,Un hombre sentado está tocando el violonchelo.,0.85,6,7,Un hombre se dedica a interpretar el violonchelo.
...,...,...,...,...,...,...
5744,Vendavales severos mientras la tormenta Clodag...,Merkel promete la solidaridad de la OTAN con L...,0.00,10,9,Fuertes ráfagas de viento mientras la tormenta...
5745,Docenas de egipcios rehenes tomados por terror...,El número de muertos en el accidente de un bar...,0.00,14,22,Decenas de egipcios son tomados como rehenes p...
5746,El Presidente se dirige a Bahrein,Presidente Xi: China seguirá ayudando a combat...,0.00,6,9,El Presidente se dirige a Bahrein
5747,China y la India se comprometen a fomentar los...,China lucha por tranquilizar a los nerviosos c...,0.00,11,10,China y la India se comprometen a fortalecer s...


## Gemini augmentation

In [4]:
from plugins.estimators import Gemini_flash15

/home/manuel.couto.pintos/Documents/code/ludosym/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
df = pd.read_csv('data/augmented_sts.csv', index_col=0)

In [6]:
df

,Unnamed: 0,sentence1,sentence2,similarity_score,sent1_len,sent2_len,aug1
0,0,Un avión está despegando.,Un avión está despegando.,1.00,4,4,Un avión se encuentra en pleno despegue.
1,1,Un hombre está tocando una gran flauta.,Un hombre está tocando una flauta.,0.76,7,6,Un individuo se dedica a interpretar una gran ...
2,2,Un hombre está untando queso rallado en una pi...,Un hombre está untando queso rallado en una pi...,0.76,9,10,Un hombre se encuentra esparciendo queso ralla...
3,3,Tres hombres están jugando al ajedrez.,Dos hombres están jugando al ajedrez.,0.52,6,6,Tres hombres se hallan disputando una partida ...
4,4,Un hombre está tocando el violonchelo.,Un hombre sentado está tocando el violonchelo.,0.85,6,7,Un hombre se dedica a interpretar el violonchelo.
...,...,...,...,...,...,...,...
5744,5744,Vendavales severos mientras la tormenta Clodag...,Merkel promete la solidaridad de la OTAN con L...,0.00,10,9,Fuertes ráfagas de viento mientras la tormenta...
5745,5745,Docenas de egipcios rehenes tomados por terror...,El número de muertos en el accidente de un bar...,0.00,14,22,Decenas de egipcios son tomados como rehenes p...
5746,5746,El Presidente se dirige a Bahrein,Presidente Xi: China seguirá ayudando a combat...,0.00,6,9,El Presidente se dirige a Bahrein
5747,5747,China y la India se comprometen a fomentar los...,China lucha por tranquilizar a los nerviosos c...,0.00,11,10,China y la India se comprometen a fortalecer s...


In [7]:
import google.generativeai as genai

g_model = Gemini_flash15()

genai.configure(api_key=os.environ["GEMINI_KEY"])

In [26]:
import numpy as np
import time
from tqdm import tqdm
system = dict(role='system', content='Te pasaré una lista de 10 textos. Parafrasea cada uno asegurandote de que el texto generado sea distinto al original. Asegurate de conservar significado. Separa cada uno de los textos generados con && No quites ni añadas información. Usa sinónimos en la medida de lo posible. En total generarás 10 textos. Uno por cada texto original.')
all_responses = []
for idx, batch in tqdm(list(df.groupby(np.arange(len(df))//10))[0:100]):
    error = True
    while error:
        try:
            response = g_model.model.send([system]+batch.sentence2.apply(lambda x: dict(role='user', content=x)).values.tolist())
            all_responses.append(response)
            error = False
            if idx % 10 == 0 and idx != 0:
                time.sleep(180)
        except Exception as ex:
            error = True
            print('Sleeping 1m....')
            time.sleep(180)


  0%|          | 0/100 [00:00<?, ?it/s]

 44%|████▍     | 44/100 [13:13<19:24, 20.79s/it]  

Sleeping 1m....
Sleeping 1m....


 89%|████████▉ | 89/100 [32:28<00:53,  4.84s/it]   

Sleeping 1m....
Sleeping 1m....


100%|██████████| 100/100 [41:50<00:00, 25.11s/it]


In [27]:
all_responses

[[{'role': 'system',
   'content': 'Te pasaré una lista de 10 textos. Parafrasea cada uno asegurandote de que el texto generado sea distinto al original. Asegurate de conservar significado. Separa cada uno de los textos generados con && No quites ni añadas información. Usa sinónimos en la medida de lo posible. En total generarás 10 textos. Uno por cada texto original.'},
  {'role': 'user', 'content': 'Un avión está despegando.'},
  {'role': 'user', 'content': 'Un hombre está tocando una flauta.'},
  {'role': 'user',
   'content': 'Un hombre está untando queso rallado en una pizza cruda.'},
  {'role': 'user', 'content': 'Dos hombres están jugando al ajedrez.'},
  {'role': 'user',
   'content': 'Un hombre sentado está tocando el violonchelo.'},
  {'role': 'user', 'content': 'Dos hombres están luchando.'},
  {'role': 'user', 'content': 'Un hombre está patinando.'},
  {'role': 'user', 'content': 'El hombre está tocando la guitarra.'},
  {'role': 'user',
   'content': 'Una mujer está tocand

In [28]:
import itertools
all_content = list(itertools.chain(*list(map(lambda x: x[-1]['content'].split('&&'), all_responses))))

In [34]:
all_content = list(map(lambda x: x.strip(), all_content))

In [37]:
df['aug2'] = all_content + [None] * (len(df.index) - len(all_content))

In [42]:
df

,Unnamed: 0,sentence1,sentence2,similarity_score,sent1_len,sent2_len,aug1,aug2
0,0,Un avión está despegando.,Un avión está despegando.,1.00,4,4,Un avión se encuentra en pleno despegue.,Una aeronave está iniciando su ascenso.
1,1,Un hombre está tocando una gran flauta.,Un hombre está tocando una flauta.,0.76,7,6,Un individuo se dedica a interpretar una gran ...,Un individuo está produciendo música con una f...
2,2,Un hombre está untando queso rallado en una pi...,Un hombre está untando queso rallado en una pi...,0.76,9,10,Un hombre se encuentra esparciendo queso ralla...,Un hombre está cubriendo una pizza sin cocinar...
3,3,Tres hombres están jugando al ajedrez.,Dos hombres están jugando al ajedrez.,0.52,6,6,Tres hombres se hallan disputando una partida ...,Dos individuos están participando en un juego ...
4,4,Un hombre está tocando el violonchelo.,Un hombre sentado está tocando el violonchelo.,0.85,6,7,Un hombre se dedica a interpretar el violonchelo.,Un hombre está sentado tocando un violonchelo.
...,...,...,...,...,...,...,...,...
5744,5744,Vendavales severos mientras la tormenta Clodag...,Merkel promete la solidaridad de la OTAN con L...,0.00,10,9,Fuertes ráfagas de viento mientras la tormenta...,None
5745,5745,Docenas de egipcios rehenes tomados por terror...,El número de muertos en el accidente de un bar...,0.00,14,22,Decenas de egipcios son tomados como rehenes p...,None
5746,5746,El Presidente se dirige a Bahrein,Presidente Xi: China seguirá ayudando a combat...,0.00,6,9,El Presidente se dirige a Bahrein,None
5747,5747,China y la India se comprometen a fomentar los...,China lucha por tranquilizar a los nerviosos c...,0.00,11,10,China y la India se comprometen a fortalecer s...,None


In [43]:
df.to_csv('data/augmented_sts_aug2_prog.csv')

In [45]:
aux_df = df.loc[df.aug2.isna()]
all_responses2 = []
for idx, batch in tqdm(list(aux_df.groupby(np.arange(len(aux_df))//10))):
    try:
        response = g_model.model.send([system]+batch.sentence2.apply(lambda x: dict(role='user', content=x)).values.tolist())
        aux = response[-1]['content'].split('&&')
        aux = list(map(lambda x: x.strip(), aux))
        all_responses2.append(aux)
        # if idx % 10 == 0 and idx != 0:
        #     time.sleep(180)
    except Exception as ex:
        print('Error....')
        all_responses2.append([None]*10)

  0%|          | 2/475 [00:03<12:12,  1.55s/it]

Error....


  1%|          | 3/475 [00:04<13:30,  1.72s/it]

Error....


  1%|          | 5/475 [00:08<14:34,  1.86s/it]

...

 91%|█████████ | 430/475 [17:18<01:35,  2.13s/it]

Error....


 91%|█████████ | 432/475 [17:21<01:12,  1.68s/it]

Error....


100%|██████████| 475/475 [18:59<00:00,  2.40s/it]


In [51]:
all_responses3 = list(map(lambda x: [None]*10 if len(x) != 10 else x, all_responses2))

In [52]:
any(map(lambda x: len(x) != 10, all_responses3))

False

In [70]:
all_responses3

[['Dos locomotoras de color verde y blanco en la vía férrea.',
  'Un felino blanco se encuentra de pie en el suelo.',
  'Un enorme navío en el océano.',
  'Un ómnibus rojo de dos niveles que recorre la calle.',
  'Un tren para viajeros se encuentra en la terminal.',
  'La mujer sentada en la mesa está absorta en algo.',
  'Una computadora Macintosh se encuentra en el suelo.',
  'Una perspectiva lateral del rostro de un equino.',
  'Un grupo de personas de edad avanzada sentadas alrededor de una mesa.',
  'Un jinete a lomos de un caballo en un cercado.'],
 [None, None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None, None],
 ['Dos mujeres de la India junto a dos niñas indias cerca del agua.',
  'Un autobús de dos niveles con pasajeros circulando entre el resto de vehículos.',
  'Vacas delgadas pastando en el campo.',
  'Un grupo de caballos pastando en la pradera.',
  'Un primer plano de un escritorio con un ordenador y papeles sob

In [74]:
df.loc[df.aug2.isna(), 'aug2'] = list(itertools.chain(*all_responses3))[:-1]

In [75]:
df.loc[~df.aug2.isna()]

,Unnamed: 0,sentence1,sentence2,similarity_score,sent1_len,sent2_len,aug1,aug2
0,0,Un avión está despegando.,Un avión está despegando.,1.00,4,4,Un avión se encuentra en pleno despegue.,Una aeronave está iniciando su ascenso.
1,1,Un hombre está tocando una gran flauta.,Un hombre está tocando una flauta.,0.76,7,6,Un individuo se dedica a interpretar una gran ...,Un individuo está produciendo música con una f...
2,2,Un hombre está untando queso rallado en una pi...,Un hombre está untando queso rallado en una pi...,0.76,9,10,Un hombre se encuentra esparciendo queso ralla...,Un hombre está cubriendo una pizza sin cocinar...
3,3,Tres hombres están jugando al ajedrez.,Dos hombres están jugando al ajedrez.,0.52,6,6,Tres hombres se hallan disputando una partida ...,Dos individuos están participando en un juego ...
4,4,Un hombre está tocando el violonchelo.,Un hombre sentado está tocando el violonchelo.,0.85,6,7,Un hombre se dedica a interpretar el violonchelo.,Un hombre está sentado tocando un violonchelo.
...,...,...,...,...,...,...,...,...
5735,5735,El presidente de Taiwán fleta un avión para pr...,El presidente de Polonia pierde ante su rival,0.00,13,8,El presidente de Taiwán alquila un avión para ...,El mandatario polaco sucumbe ante su contrincante
5736,5736,Dos palestinos mueren en ataques a israelíes a...,Un funcionario palestino critica el proyecto d...,0.00,13,15,Dos palestinos fallecen en atentados contra is...,Un representante palestino censura la norma de...
5737,5737,Thomas Cook acusado de poner los costos antes ...,La fraternidad de la Universidad de Florida ac...,0.00,11,16,Thomas Cook es acusado de priorizar los costes...,Una fraternidad de la Universidad de Florida e...
5738,5738,"Narendra Modi ""photoshopeó"" la imagen de la vi...",El Primer Ministro Narendra Modi habla de la n...,0.00,16,13,"La imagen ""editada digitalmente"" de la visita ...",El primer ministro Narendra Modi habla sobre l...


In [80]:
df.drop(columns=['sent1_len', 'sent2_len']).to_csv('data/augmented_sts_aug2_prog.csv')

In [79]:
df

,Unnamed: 0,sentence1,sentence2,similarity_score,sent1_len,sent2_len,aug1,aug2
0,0,Un avión está despegando.,Un avión está despegando.,1.00,4,4,Un avión se encuentra en pleno despegue.,Una aeronave está iniciando su ascenso.
1,1,Un hombre está tocando una gran flauta.,Un hombre está tocando una flauta.,0.76,7,6,Un individuo se dedica a interpretar una gran ...,Un individuo está produciendo música con una f...
2,2,Un hombre está untando queso rallado en una pi...,Un hombre está untando queso rallado en una pi...,0.76,9,10,Un hombre se encuentra esparciendo queso ralla...,Un hombre está cubriendo una pizza sin cocinar...
3,3,Tres hombres están jugando al ajedrez.,Dos hombres están jugando al ajedrez.,0.52,6,6,Tres hombres se hallan disputando una partida ...,Dos individuos están participando en un juego ...
4,4,Un hombre está tocando el violonchelo.,Un hombre sentado está tocando el violonchelo.,0.85,6,7,Un hombre se dedica a interpretar el violonchelo.,Un hombre está sentado tocando un violonchelo.
...,...,...,...,...,...,...,...,...
5744,5744,Vendavales severos mientras la tormenta Clodag...,Merkel promete la solidaridad de la OTAN con L...,0.00,10,9,Fuertes ráfagas de viento mientras la tormenta...,None
5745,5745,Docenas de egipcios rehenes tomados por terror...,El número de muertos en el accidente de un bar...,0.00,14,22,Decenas de egipcios son tomados como rehenes p...,None
5746,5746,El Presidente se dirige a Bahrein,Presidente Xi: China seguirá ayudando a combat...,0.00,6,9,El Presidente se dirige a Bahrein,None
5747,5747,China y la India se comprometen a fomentar los...,China lucha por tranquilizar a los nerviosos c...,0.00,11,10,China y la India se comprometen a fortalecer s...,None


In [83]:
df2 = pd.read_csv('data/augmented_sts_aug2_prog.csv', index_col=0)

In [84]:
df2.loc[~df2.aug2.isna()]

,sentence1,sentence2,similarity_score,aug1,aug2
0,Un avión está despegando.,Un avión está despegando.,1.00,Un avión se encuentra en pleno despegue.,Una aeronave está iniciando su ascenso.
1,Un hombre está tocando una gran flauta.,Un hombre está tocando una flauta.,0.76,Un individuo se dedica a interpretar una gran ...,Un individuo está produciendo música con una f...
2,Un hombre está untando queso rallado en una pi...,Un hombre está untando queso rallado en una pi...,0.76,Un hombre se encuentra esparciendo queso ralla...,Un hombre está cubriendo una pizza sin cocinar...
3,Tres hombres están jugando al ajedrez.,Dos hombres están jugando al ajedrez.,0.52,Tres hombres se hallan disputando una partida ...,Dos individuos están participando en un juego ...
4,Un hombre está tocando el violonchelo.,Un hombre sentado está tocando el violonchelo.,0.85,Un hombre se dedica a interpretar el violonchelo.,Un hombre está sentado tocando un violonchelo.
...,...,...,...,...,...
5735,El presidente de Taiwán fleta un avión para pr...,El presidente de Polonia pierde ante su rival,0.00,El presidente de Taiwán alquila un avión para ...,El mandatario polaco sucumbe ante su contrincante
5736,Dos palestinos mueren en ataques a israelíes a...,Un funcionario palestino critica el proyecto d...,0.00,Dos palestinos fallecen en atentados contra is...,Un representante palestino censura la norma de...
5737,Thomas Cook acusado de poner los costos antes ...,La fraternidad de la Universidad de Florida ac...,0.00,Thomas Cook es acusado de priorizar los costes...,Una fraternidad de la Universidad de Florida e...
5738,"Narendra Modi ""photoshopeó"" la imagen de la vi...",El Primer Ministro Narendra Modi habla de la n...,0.00,"La imagen ""editada digitalmente"" de la visita ...",El primer ministro Narendra Modi habla sobre l...


In [91]:
system = dict(role='system', content='Te pasaré un texto. Parafrasealo asegurandote de que el texto generado sea distinto al original. Asegurate de conservar significado. No quites ni añadas información. Usa sinónimos en la medida de lo posible. Genera un único texto por cada texto original.')
all_responses4 = []

for sent in tqdm(df2.loc[df2.aug2.isna()].sentence2.tolist()):
    error = True
    cumm_errors = 0
    while error:
        try:
            response = g_model.model.send([system, dict(role='user', content=sent)])
            aux = response[-1]['content']
            all_responses4.append(aux)
            error = False
        except Exception:
            error = True
            cumm_errors += 1
            print('Error....')
            if cumm_errors > 10:
                print(sent)
                all_responses4.append(None)
                cumm_errors = 0
                error = False    

            


  2%|▏         | 16/1029 [00:09<10:35,  1.59it/s]

Error....


  2%|▏         | 20/1029 [00:12<11:12,  1.50it/s]

Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....


  2%|▏         | 21/1029 [00:19<41:36,  2.48s/it]

...

 99%|█████████▉| 1019/1029 [16:00<00:06,  1.48it/s]

Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....


 99%|█████████▉| 1020/1029 [16:03<00:11,  1.28s/it]

Error....
Error....
Palestino arrestado por acosar sexualmente a jóvenes


100%|██████████| 1029/1029 [16:09<00:00,  1.06it/s]


In [98]:
list(map(lambda x: x.strip() if x is not None else x, all_responses4))

'Un tornado reciente en Texas ha resultado en una víctima mortal.'

In [99]:
df2.loc[df2.aug2.isna(), 'aug2'] = list(map(lambda x: x.strip() if x is not None else x, all_responses4))

In [102]:
df2.to_csv('data/augmented_sts_aug2_prog.csv')

In [105]:
system = dict(role='system', content='Te pasaré un texto. Parafrasealo asegurandote de que el texto generado sea distinto al original. Asegurate de conservar significado. No quites ni añadas información. Usa sinónimos en la medida de lo posible. Genera un único texto por cada texto original.')
all_responses5 = []

for sent in tqdm(df2.loc[df2.aug2.isna()].sentence2.tolist()):
    error = True
    cumm_errors = 0
    while error:
        try:
            response = g_model.model.send([system, dict(role='user', content=sent)])
            aux = response[-1]['content']
            all_responses5.append(aux)
            error = False
        except Exception:
            error = True
            cumm_errors += 1
            print('Error....')
            if cumm_errors > 100:
                print(sent)
                all_responses5.append(None)
                cumm_errors = 0
                error = False    


  0%|          | 0/38 [00:00<?, ?it/s]

Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....


  3%|▎         | 1/38 [01:00<37:14, 60.39s/it]

Error....
Dos mujeres sentadas afuera riéndose.
Error....
Error....
Error....
Error....
Error....


  5%|▌         | 2/38 [01:04<16:21, 27.26s/it]

...

Error....
Turistas estadounidenses violados en el norte de la India: Policía
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Err

 97%|█████████▋| 37/38 [27:31<01:00, 60.64s/it]

Error....
CCTV: Los estafadores de York se visten como mujeres para engañar a los pensionistas
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error....
Error

100%|██████████| 38/38 [27:53<00:00, 44.03s/it]

Error....
Palestino arrestado por acosar sexualmente a jóvenes


In [108]:
df2.loc[df2.aug2.isna(), 'aug2'] = list(map(lambda x: x.strip() if x is not None else x, all_responses5))

In [109]:
df2.loc[df2.aug2.isna()]

,sentence1,sentence2,similarity_score,aug1,aug2
1040,Una mujer sonriente con una cerveza sentada af...,Dos mujeres sentadas afuera riéndose.,0.6400,Una dama con una expresión alegre disfruta de ...,None
1701,Un hombre y una mujer mirando a la cámara.,Un hombre y una mujer riéndose.,0.4660,Un hombre y una mujer dirigiendo sus miradas h...,None
1767,Un hombre y una mujer están sonriendo.,Un hombre y una mujer riéndose.,0.6400,Un hombre y una mujer muestran una expresión d...,None
1837,Un hombre y una mujer riéndose.,Un hombre y una mujer se besan.,0.2400,Un hombre y una mujer se ríen a carcajadas.,None
1887,Un perro con manchas blancas y negras salta so...,Un perro con manchas blancas y negras está sal...,0.9200,Un can con manchas blancas y negras brinca sob...,None
1927,Una dama vestida con un kimono está mirando a ...,Dos chicas vestidas de negro están sentadas en...,0.2000,Una dama vestida con un kimono mira hacia la c...,None
2364,Florida La ignorancia es tuya!,¡La ignorancia es suya!,0.6800,"¡Tu ignorancia es propia, Florida!",None
2368,Y Nugent fue un completo tonto al decir lo que...,Natalie fue una completa tonta por haber insul...,0.1200,Y Nugent fue un completo necio al expresar lo ...,None
2717,18 de julio: Hurlbert presenta un solo cargo d...,"Bryant, de 24 años, ha sido acusado de un deli...",0.6112,18 de julio: Hurlbert formula un único cargo d...,None
2719,"En cambio, Weida decidió que el caso de Meeste...","Douglas Meester, de 20 años, está acusado de v...",0.6500,"Sin embargo, Weida determinó que el caso de Me...",None


In [115]:
system = dict(role='system', content='Te pasaré un texto. Parafrasealo asegurandote de que el texto generado sea distinto al original. Asegurate de conservar significado. No quites ni añadas información. Usa sinónimos en la medida de lo posible. Genera un único texto por cada texto original.')
all_responses6 = []

for sent in tqdm(df2.loc[df2.aug2.isna()].sentence2.tolist()):
    error = True
    cumm_errors = 0
    while error:
        try:
            response = model.model.send([system, dict(role='user', content=sent)])
            aux = response[-1]['content']
            all_responses6.append(aux)
            error = False
        except Exception:
            error = True
            cumm_errors += 1
            print('Error....')
            if cumm_errors > 100:
                print(sent)
                all_responses6.append(None)
                cumm_errors = 0
                error = False    

  4%|▎         | 1/27 [00:05<02:21,  5.43s/it]

index=0 message=ChatMessage(role='assistant', content='Dos damas se encuentran sentadas al aire libre compartiendo una gran carcajada.', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>


  7%|▋         | 2/27 [00:06<01:13,  2.96s/it]

index=0 message=ChatMessage(role='assistant', content='Un caballero y una dama riendo con júbilo.', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>


 11%|█         | 3/27 [00:11<01:31,  3.79s/it]

...

index=0 message=ChatMessage(role='assistant', content='En el norte de la India, visitantes estadounidenses sufrieron una agresión sexual: Fuerzas del orden.', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>


 96%|█████████▋| 26/27 [01:24<00:03,  3.64s/it]

index=0 message=ChatMessage(role='assistant', content='Las impostoras de York se disfrazan de mujeres para embaucar a los jubilados en sistemas de vigilancia CCTV.', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>


100%|██████████| 27/27 [01:30<00:00,  3.34s/it]

index=0 message=ChatMessage(role='assistant', content='Se detuvo a un individuo de origen palestino por hostigar sexualmente a personas jóvenes.', name=None, tool_calls=None, tool_call_id=None) finish_reason=<FinishReason.stop: 'stop'>


In [117]:
df2.loc[df2.aug2.isna(), 'aug2'] = list(map(lambda x: x.strip() if x is not None else x, all_responses6))

In [118]:
df2.loc[df2.aug2.isna()]

,sentence1,sentence2,similarity_score,aug1,aug2


In [119]:
df2.to_csv('data/augmented_sts_aug12.csv')

In [120]:
df2

,sentence1,sentence2,similarity_score,aug1,aug2
0,Un avión está despegando.,Un avión está despegando.,1.00,Un avión se encuentra en pleno despegue.,Una aeronave está iniciando su ascenso.
1,Un hombre está tocando una gran flauta.,Un hombre está tocando una flauta.,0.76,Un individuo se dedica a interpretar una gran ...,Un individuo está produciendo música con una f...
2,Un hombre está untando queso rallado en una pi...,Un hombre está untando queso rallado en una pi...,0.76,Un hombre se encuentra esparciendo queso ralla...,Un hombre está cubriendo una pizza sin cocinar...
3,Tres hombres están jugando al ajedrez.,Dos hombres están jugando al ajedrez.,0.52,Tres hombres se hallan disputando una partida ...,Dos individuos están participando en un juego ...
4,Un hombre está tocando el violonchelo.,Un hombre sentado está tocando el violonchelo.,0.85,Un hombre se dedica a interpretar el violonchelo.,Un hombre está sentado tocando un violonchelo.
...,...,...,...,...,...
5744,Vendavales severos mientras la tormenta Clodag...,Merkel promete la solidaridad de la OTAN con L...,0.00,Fuertes ráfagas de viento mientras la tormenta...,"La canciller alemana, Merkel, garantiza el apo..."
5745,Docenas de egipcios rehenes tomados por terror...,El número de muertos en el accidente de un bar...,0.00,Decenas de egipcios son tomados como rehenes p...,El recuento de fallecidos en el naufragio de u...
5746,El Presidente se dirige a Bahrein,Presidente Xi: China seguirá ayudando a combat...,0.00,El Presidente se dirige a Bahrein,"El líder chino, Xi Jinping, ha reiterado el co..."
5747,China y la India se comprometen a fomentar los...,China lucha por tranquilizar a los nerviosos c...,0.00,China y la India se comprometen a fortalecer s...,Las autoridades chinas se esfuerzan en reconfo...
